# **FINAL CHALLENGE: LLM APPLICATION FOR MEDICALLY FINE TUNED CHATBOT**

*   Enrique Almazán
*   Victor Miguel Álvarez Camarero
*   Javier Alfonso Villoldo

# **OBJECTIVES**

The main objective of this challenge is to create and validate a solution for potential applications aimed at providing medical knowledge and assistance.

Our solution consists of filtering medical conversations from the provided 'dataset4FinalChallenge.snappy.parquet' dataset subset by leveraging the course's tools and techniques, taking into account that the solution should optimally handle a large dataset of 1 TB. These conversations then serve as the foundation for fine-tuning a Language Model (LLM), a sophisticated AI-driven tool capable of understanding and generating human-like text.

The resulting application holds immense potential for bridging the gap in healthcare accessibility, particularly situations where access to health resources is limited. Users will gain access to a reliable and user-friendly platform offering personalized medical guidance based on real-world conversations between patients and healthcare professionals. Not only that but this application would also help foster a deeper understanding of medical issues, empowering individuals to make informed decisions about their well-being.

In this way, we can better understand the concerns, uncertainties, and questions that people have about their health, as well as provide medical guidance in response.

In [1]:
# Install libraries
!pin install datasets
!pip install accelerate
!pip install bitsançdbytes
!pip install peft
!pip install evaluate
!pip install trl
!pip install rouge_score
!pip install pyspark[sql]
!pip install gdown
!pip install pyspark

/bin/bash: line 1: pin: command not found
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cach

In [2]:
# The following libraries were required to properly load the model
! pip install accelerate
! pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.6 MB/s eta 0:00:00


# **DEPENDENCIES**

For the development and implementation of our solution we rely on a set of essential dependencies. Here's a brief overview of the main ones:

- **gdown**: to efficiently load the dataset file from Google Drive, streamlining the data retrieval process directly into our Google Collab environment.

- **Pyspark**: to process the dataset efficiently. Leveraging its distributed computing framework, we can handle large-scale data manipulation tasks with ease, ensuring optimal performance.

- **Huggingface_hub**: this dependency enables us to access and load pre-trained language models seamlessly. By leveraging the huggingface_hub, we can utilize state-of-the-art models for our task without the need for manual downloading and configuration.

- **Transformers** and **trl**: these libraries are essential for fine-tuning language models to our specific task. By utilizing transformers and trl, we can train and customize models to accurately capture the concerns, uncertainties, and questions that people have about their health, as well as learn to provide medical guidance in response.

Additionally, it's worth noting that for the development of the code, a Google Colab shared file has been utilized.

In [3]:
# Libraries for loading the data file
import gdown

# Libraries for processing the data
from __future__ import print_function
from functools import wraps
import pyspark as spark
from pyspark import SparkConf
import time
from operator import add
import os
from subprocess import STDOUT, check_call, check_output
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col, udf, desc, count, expr, when, array_contains
from pyspark.sql.types import StringType

# Librarires for Fine tuning the LLM
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, pipeline
from datasets import Dataset
from peft import LoraConfig, TaskType, get_peft_model
from huggingface_hub import notebook_login
import evaluate
from trl import SFTTrainer
import requests
import json

#**1. LOAD THE DATA**

In this initial phase of our analysis, we start on acquiring the necessary data files residing in the Google Drive cloud. Using the `gdown` library, we retrieve the dataset effortlessly. The Parquet-compressed file encapsulates the physician-patient medical conversations in a structured and efficient manner, facilitating seamless data ingestion and processing within our Spark environment.

In this section we also lay the foundation for our subsequent analyses, by starting Spark and configuring a function `set_conf()` that sets-up the Spark application's settings.

In [4]:
# Download the data
url1 = 'https://drive.google.com/uc?id=1O7z5rDGLTd_YfRXd7_EImqAgr4Pmph1R'
url2 = 'https://drive.google.com/uc?id=1A5FkmQyTrsBKGHs7k6FWHnTSOnmyt5kI'
output1 = 'part00000'
output2 = 'part00001'

gdown.download(url1, output1, quiet=False)
gdown.download(url2, output2, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1O7z5rDGLTd_YfRXd7_EImqAgr4Pmph1R
To: /content/part00000
100%|██████████| 7.93M/7.93M [00:00<00:00, 74.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1A5FkmQyTrsBKGHs7k6FWHnTSOnmyt5kI
To: /content/part00001
100%|██████████| 8.07M/8.07M [00:00<00:00, 66.4MB/s]


'part00001'

In [5]:
# Initialize Spark
def set_conf():
    conf = SparkConf().setAppName("App")
    conf = (conf.setMaster('local[*]')
      .set('spark.executor.memory', '4G')
      .set('spark.driver.memory', '16G')
      .set('spark.driver.maxResultSize', '8G'))
    return conf

In [6]:
# Create a SparkSession
spark = SparkSession.builder \
    .appName("Final Challenge (Medical LLM chatbot)") \
    .getOrCreate()

/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


# **2. INFORMATION ABOUT THE DATA**

The data's structure, types, and metadata is examined here. The data types, as well as the dataframe it self are displayed to visually grasp its contents.

Furthermore, we check that both files have the same structure in order to merge them for future pre-processing and training. This analysis lays the groundwork for deeper exploration and filtering of the medical conversations of interest.

In [7]:
# Read the downloaded Parquet files into a DataFrame
df1 = spark.read.parquet("part00000")
df2 = spark.read.parquet("part00001")

In [8]:
# Strucure of our DataFrame (check if both files have the same structure)
display(df1)
display(df2)

# Data types of each column
df1.dtypes
df2.dtypes

DataFrame[ID: double, CreationDate: timestamp, TextData: string]

DataFrame[ID: double, CreationDate: timestamp, TextData: string]

[('ID', 'double'), ('CreationDate', 'timestamp'), ('TextData', 'string')]

In [9]:
# Merge DataFrames
merged_df = df1.union(df2)

# Show the resulting DataFrame
merged_df.show()

+--------------------+--------------------+--------------------+
|                  ID|        CreationDate|            TextData|
+--------------------+--------------------+--------------------+
|  0.5577925291272189|2024-05-10 20:05:...|instruction=Hi, d...|
| 0.14286049460007832| 2023-08-31 00:00:00|jurisdiction=UNKN...|
|  0.5670049908907079|2024-05-10 20:05:...|instruction=Docto...|
|0.031952793120539336|2024-05-10 20:05:...|instruction=Docto...|
| 0.18870965764235437|2024-05-10 20:05:...|instruction=Yes, ...|
|  0.5448709062977307| 2023-08-31 00:00:00|jurisdiction=EU|i...|
| 0.32692921200808733|2024-05-10 20:05:...|instruction=Thank...|
| 0.24083509068287046| 2023-08-31 00:00:00|jurisdiction=EU|i...|
|  0.7190659500061238|2024-05-10 20:05:...|instruction=Hi, s...|
|  0.7133354174966207|2024-05-10 20:05:...|instruction=Docto...|
|  0.4168523612930384|2024-05-10 20:05:...|instruction=Docto...|
| 0.42590747308489274|2024-05-10 20:05:...|instruction=Hi, s...|
|  0.6436637212750892|202

In [10]:
# Lets print some samples (rows) for our merged dataframe
merged_df.show(n=1, truncate=False)
merged_df.show(n=2, truncate=False)
merged_df.show(n=3, truncate=False)

+------------------+--------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ID                |CreationDate              |TextData                                                                                                                                                                                                                                                                                                                                                                                                                      

#**3. PRE-PROCESSING**

We will preserve the input and the outputs for exclusively medical-related conversations, which are those that do not present a jurisdiction label.

In this section, we preprocess the data from the *TextData* column to extract the *'instructions'* and the *'answers'* subcolumns, that contain the relevant medical conversations. Plus, since we are only interested in medical conversations we will filter out unwanted text that has the *'jurisdiction'* key, only applicable to legal data.

In order to do so we define regular expressions. Regular expressions are patterns used to match character combinations in strings. We believe this method of extraction of specific information is efficient and easily applicable to large datasets (1TB).

In [11]:
from pyspark.sql import functions as F

# Filter conversations without a jurisdiction label
filtered_df = merged_df.filter(~F.col("TextData").contains("jurisdiction"))

In [12]:
filtered_df.show()

num_rows = filtered_df.count()
print("Número de filas en el DataFrame:", num_rows)

+--------------------+--------------------+--------------------+
|                  ID|        CreationDate|            TextData|
+--------------------+--------------------+--------------------+
|  0.5577925291272189|2024-05-10 20:05:...|instruction=Hi, d...|
|  0.5670049908907079|2024-05-10 20:05:...|instruction=Docto...|
|0.031952793120539336|2024-05-10 20:05:...|instruction=Docto...|
| 0.18870965764235437|2024-05-10 20:05:...|instruction=Yes, ...|
| 0.32692921200808733|2024-05-10 20:05:...|instruction=Thank...|
|  0.7190659500061238|2024-05-10 20:05:...|instruction=Hi, s...|
|  0.7133354174966207|2024-05-10 20:05:...|instruction=Docto...|
|  0.4168523612930384|2024-05-10 20:05:...|instruction=Docto...|
| 0.42590747308489274|2024-05-10 20:05:...|instruction=Hi, s...|
|  0.6436637212750892|2024-05-10 20:05:...|instruction=There...|
|  0.3983200542792885|2024-05-10 20:05:...|instruction=Docto...|
| 0.23502625006504996|2024-05-10 20:05:...|instruction=Docto...|
|  0.4806425726213206|202

In [13]:
from pyspark.sql.functions import col, regexp_extract

# Define regular expressions for extracting instructions and answers
input_regex = r'instruction=([^|]+)'
answer_regex = r'answer=([^|]+)'

# Extract instructions and answers from TextData column
df = filtered_df.withColumn("instruction", regexp_extract(col("TextData"), input_regex, 1)) \
       .withColumn("answer", regexp_extract(col("TextData"), answer_regex, 1))

# Display the extracted instruction and answer columns
data = df.select("instruction", "answer")

In [14]:
# Print instruction and answer columns
data.show()

+--------------------+--------------------+
|         instruction|              answer|
+--------------------+--------------------+
|Hi, doctor. Looks...|It's good to star...|
|Doctor, there's b...|    His B.P.'s high.|
|Doctor, there's a...|The symptoms sugg...|
|Yes, we are alter...|if his pain is ba...|
|Thank you! I appr...|Mitral valve is v...|
|Hi, sir, there's ...|Yes, I think you'...|
|Doctor, severe ch...|In view of your s...|
|Doctor, you've go...|The open head inj...|
|Hi, sir, your che...|In the case of Ti...|
|There are symptom...|The symptoms are ...|
|Doctor, I think s...|The drug recommen...|
|Doctor, I feel a ...|We recommend radi...|
|Doctor, there's v...|This could be a s...|
|I (23F) keep slee...|Taking a sedative...|
|There's a teacher...|According to your...|
|Hi doctor, I rece...|As a result of th...|
|Doctor, I'm losin...|The symptoms may ...|
|Doctor, it looks ...|The symptoms sugg...|
|Can CT Scan witho...|To answer your qu...|
|Doctor, I've been...|The sympto

From the column format within the DataFrame we will use a loop to extract and appropiatelly format the text that will correspond to the inputs and the outputs of our model. These will be stored in a list of dictionaries for easier handling.

Llama 2 chat models require specific formatting needs to be followed:

```<s>[INST] {user_message_1} [/INST] {model_reply_1}</s>```

In [15]:
# Initialize an empty list to store dictionaries
dict_list = []

# Iterate through each row in the DataFrame 'data'
for row in data.collect():
    # Extract 'instruction' and 'answer' from the row
    instruction = row["instruction"]
    answer = row["answer"]

    # Construct the dictionary with the required format
    text_dict = {
        "text": f"<s>[INST]{instruction}[/INST]{answer}</s>"
    }

    # Append the dictionary to the list
    dict_list.append(text_dict)

In [16]:
dict_list[0]

{'text': "<s>[INST]Hi, doctor. Looks like post-op infection.[/INST]It's good to start taking antibiotics based on symptoms. It will help you to combat infection by using vancomycin and necrosis. It will also help you to treat your wounds by using iodine, ciachlorate sodium, and mafeamphin. In case you need to, you can also use acelosomes to reduce swelling and pain. However, if you see signs of infection, you can also treat it with sepuloxide to improve your blood flow and blood flow.</s>"}

In [17]:
notebook_login()

Now we use the model's tokenizer to find and filter those instructions and answers that are larger than 70 tokens, since they size of the context window the main limiter of memory usage.

In [18]:
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

# Function that checks if the instruction and answer length exceeds 70 tokens
def filter_instructions(text):
    instruction = text.split('[INST]')[1].split('[/INST]')[0].strip()  # Extract instruction text
    answer = text.split('[/INST]')[1].strip()
    instruction_tokens = tokenizer.tokenize(instruction)  # Tokenize instruction text
    answer_tokens = tokenizer.tokenize(answer)  # Tokenize answer text
    return len(instruction_tokens) <= 70 and len(answer_tokens) <= 70

# Filter out prompts longer than 70 tokens and create a new list of dictionaries
filtered_dict_list = []
for item in dict_list:
    if filter_instructions (item['text']):
        filtered_dict_list.append(item)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [19]:
# Compare the amount of prompts that we are left with
print(len(filtered_dict_list)) # number of prompts after filtering
print(len(dict_list)) # number of prompts before filtering

4756
6307


In [20]:
# Function to extract instruction text
def get_instruction_length(text):
    # Extract instruction text within [INST] tags
    instruction_text = text.split('[INST]')[1].split('[/INST]')[0].strip()
    # Calculate length of instruction text
    instruction_length = len(instruction_text)
    return instruction_length

# Print the length of the strings to show what 70 tokens corresponds to
for item in filtered_dict_list:
  print(get_instruction_length(item['text']))

47
56
83
94
101
78
82
49
119
105
99
86
62
166
33
108
153
66
107
83
129
91
80
137
89
115
116
134
112
84
79
123
126
59
172
129
52
52
163
51
129
62
150
188
87
84
81
93
176
83
52
166
178
76
203
49
120
124
121
105
81
102
100
110
46
86
76
149
114
113
96
146
100
96
127
70
138
118
84
127
158
171
82
49
77
70
148
135
40
95
56
100
116
133
90
98
58
142
169
126
125
158
100
89
150
92
95
133
116
83
112
119
140
68
104
102
77
143
61
88
170
103
59
62
89
81
80
123
92
85
75
68
64
39
73
158
43
193
49
127
119
143
127
144
79
111
131
140
242
131
111
110
74
127
191
76
85
91
113
125
27
114
64
118
177
125
68
47
116
185
78
93
112
33
98
88
133
246
75
76
40
94
145
132
89
112
63
175
76
82
83
107
105
90
76
135
181
142
148
131
112
166
66
79
137
121
95
35
108
60
144
114
160
184
115
74
138
92
66
98
83
117
180
154
61
117
44
98
120
67
105
133
65
134
43
119
75
99
115
104
103
160
109
93
117
136
149
64
138
111
61
101
138
153
43
91
47
105
89
76
128
53
139
122
96
85
27
134
92
110
112
63
7
145
57
177
80
125
52
107
65
114
106
84

In [21]:
from sklearn.model_selection import train_test_split

# Split the list of dictionaries into training and testing sets
train_data, test_data = train_test_split(filtered_dict_list, test_size=0.2, random_state=42)

# Display the number of samples in each set
print("Training set size:", len(train_data))
print("Testing set size:", len(test_data))

Training set size: 3804
Testing set size: 952


In [22]:
print(train_data[0])
print(test_data[0])

{'text': "<s>[INST]Doctor, there's pain in the testicle. Can you tell me what's causing it?[/INST]The cause of the pain is hemochromatosis, which is a condition in which we absorb too much iron from the food we eat to cause iron overload in the tissues of the body, including the testes.</s>"}
{'text': "<s>[INST]Hi, doctor, I'm having a sore throat and a sore nose and sore throat. Can you tell me what's causing it?[/INST]We need to do some tests to make sure we're clear, but the infection is likely to cause pus sacs in the back of the throat.</s>"}


Due to the limited availability of GPU resources in Google Colab, each session is typically restricted to approximately 3 hours. With training alone consuming nearly 1 hour, we encountered frequent resource depletion before evaluation completion. Consequently, we were compelled to decrease the evaluation sample size to 70 samples to mitigate this issue. Here we randomly select them:

In [23]:
import random

# grabbing a reduced number of test samples to evaluate because of computational limitations
test_data_reduced = random.sample(test_data, k=70)
print("Testing set reduxed size:", len(test_data_reduced))

Testing set reduxed size: 70


We are now transforming the data format from a list of dictionaries to a Dataset format, ensuring it aligns correctly with the model's requirements.

In [24]:
dataset_train = Dataset.from_list(train_data)

In [25]:
dataset_train

Dataset({
    features: ['text'],
    num_rows: 3804
})

In [26]:
dataset_train[0]
type(dataset_train)

datasets.arrow_dataset.Dataset

In [27]:
dataset_test = Dataset.from_list(test_data_reduced) # HuggingFace Dataset

In [28]:
type(dataset_test)

datasets.arrow_dataset.Dataset

# **4. PRE-TRAINING**

Prior to training, it's essential to configure the tokenizer, download the model, and ensure the availability of CUDA resources. Additionally, clearing the CUDA cache is necessary to prevent potential issues when executing subsequent code.

In [29]:
# Clean the GPU cache before traning to avoid memory errors
torch.cuda.empty_cache()

In [30]:
# Check if CUDA (GPU) is available
# For processing with GPU instead of CPU
if torch.cuda.is_available():
    # Get the number of available CUDA devices
    num_devices = torch.cuda.device_count()
    print("Number of available CUDA devices:", num_devices)

    # Iterate over CUDA devices and print their indices and names
    for i in range(num_devices):
        print("GPU index", i, ":", torch.cuda.get_device_name(i))
else:
    print("CUDA is not available. CPU will be used.")

Number of available CUDA devices: 1
GPU index 0 : Tesla T4


Prepare the tokenizer for use with the specified LLM model by loading the tokenizer, configuring padding settings, and ensuring consistency in tokenization.

In [31]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf") # Load tokenizer, specify the one for the corresponding LLM model
tokenizer.pad_token = tokenizer.eos_token # Padding token of the tokenizer to be the same as the end-of-sequence (eos) token
tokenizer.padding_side = "right" # Padding should be added to the right side of the input sequences

In [32]:
# Base model
# We will use Meta Llama's Llama-2-7b-chat-hf as a base model, which is the same as the original, but easily accessible.
# Model: https://huggingface.co/meta-llama/Llama-2-7b-chat-hf

# Create quantization config (reduce precision as well as size)
# https://huggingface.co/docs/transformers/main_classes/quantization
# Quantization techniques reduce memory and computational costs
# by representing weights and activations with lower-precision data types
quantization_config = BitsAndBytesConfig(
    Load_in_4bit=True, # This flag is used to enable 4-bit quantization
    bnb_4bit_compute_dtype=torch.float16, # This sets the computational type: once the weights are loaded in 4-bit, the computations will be performed using 16-bit floating-point precision.
    bnb_4bit_quant_type="nf4" # This sets the quantization data type
)

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", # Load model
                                             quantization_config= quantization_config, # Quantification configuration
                                             device_map=0 # device_map = 0 means put the whole model on GPU 0; device_map="auto" compute the most optimized `device_map` automatically
)

Unused kwargs: ['Load_in_4bit']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

#**5. TRAINING**

Now we use the formatted input and outputs to fine tune the model using the LoRA configuration.

**- LoRA:** is a fine-tuning technique for large language models. It involves training additional "relevance parameters" alongside the main model parameters. These relevance parameters determine the importance or relevance of each layer's contribution to the final prediction. By adjusting these parameters, the model learns which layers are more relevant for the task at hand, enabling it to focus more on important parts of the input data. LoRA fine-tuning optimizes the entire model, including both the main parameters and the relevance parameters. In other words, it trains weights over each of the existing layer to train the model for an specific task indentifying more relevant layers for that task

In [33]:
# Create LoRA config
# More info in https://huggingface.co/docs/peft/main/en/conceptual_guides/lora
peft_config = LoraConfig(
    r=8, # The rank of the update matrices, expressed in int. Lower rank results in smaller update matrices with fewer trainable parameters.
    target_modules=["g_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"], # The modules to apply the LoRA update matrices.
    bias="none", # Specifies if the bias parameters should be trained.
    task_type = TaskType.CAUSAL_LM
)


In [34]:
# Subset of the arguments thath we use to the training.
# https://huggingface.co/docs/transformers/main_classes/trainer

training_params = TrainingArguments(
    output_dir="./results", # where the model's checkpoints and predictions will be stored
    num_train_epochs=1, # number of epochs
    per_device_train_batch_size=4, # batch size for training
    gradient_accumulation_steps=1, # # Number of update steps to accumulate the gradients for
    optim="paged_adamw_32bit", # AdamW optimizer
    save_steps=25, # save checkpoint every 25 update steps
    logging_steps=25, # logs every 25 update steps
    learning_rate=2e-4, # initial learning rate
    weight_decay=0.001, # weight decay to apply to all layers except bias/LayerNorm weights
    fp16=False,
    bf16=False,
    max_grad_norm=0.3, # maximum gradient normal (gradient clipping)
    max_steps=-1, # number of training steps (if not -1 overrides num_train_epochs)
    warmup_ratio=0.03, # ratio of steps for a linear warmup (from 0 to learning rate)
    group_by_length=True, # group sequences into batches with same length
    lr_scheduler_type="constant", # learning rate schedule
    report_to="tensorboard"
)

In [35]:
# Set supervised fine-tuning parameters
max_seq_length = None
packing = False

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_train,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_params,
    packing=packing,
)

# Train model
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/3804 [00:00<?, ? examples/s]

Step,Training Loss
25,2.668700
50,1.753600
75,1.878000
100,1.495400
125,1.748200
150,1.462500
175,1.686000
200,1.468400
225,1.692100
250,1.466900


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

TrainOutput(global_step=951, training_loss=1.5662318920612837, metrics={'train_runtime': 3083.8808, 'train_samples_per_second': 1.234, 'train_steps_per_second': 0.308, 'total_flos': 1.1467804671148032e+16, 'train_loss': 1.5662318920612837, 'epoch': 1.0})

#**6. EVALUATION**

In [36]:
# Evaluate the Model Quantitatively
rouge = evaluate.load('rouge') # https://en.wikipedia.org/wiki/ROUGE_(metric)

In [37]:
# Isolate test inputs for the model to generate a response
input = []
for d in dataset_test['text']:
  instruction_text = d.split('[INST]')[1].split('[/INST]')[0].strip()
  input.append(f"<s>[INST] {instruction_text} [/INST]")

In [38]:
# Show input format
print(input[0])
print(input[10])

<s>[INST] Doctor, I'm really distressed by the strange skin spots on my fingers and feet. [/INST]
<s>[INST] Doctor, I'm experiencing an involuntary urination, a blush, a night BB, and other prostate-related symptoms. What's the cause of these symptoms? [/INST]


In [39]:
# Isolate test outputs for later evaluation
output = dataset_test['text']
print(output[0])
print(output[10])

<s>[INST]Doctor, I'm really distressed by the strange skin spots on my fingers and feet.[/INST]It's a common skin disease caused by an HPV virus.</s>
<s>[INST]Doctor, I'm experiencing an involuntary urination, a blush, a night BB, and other prostate-related symptoms. What's the cause of these symptoms?[/INST]It's very likely that it's prostate cancer, and you have to run some tests to make sure you're diagnosed, and it's important to find out early so that you can start treatment as soon as possible.</s>


In [40]:
# Define the text generation pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

output_model = []

for i in input:
  output_model.append(pipe(i))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [41]:
# Show examples of generated outputs
print(output_model[0])
print(output_model[10])

[{'generated_text': "<s>[INST] Doctor, I'm really distressed by the strange skin spots on my fingers and feet. [/INST] The symptoms suggest that you're suffering from a fungus infection. Have you been exposed to any new environments recently? Patient: Yes, I've been to a new location recently. Doctor: Okay, let's run some tests to check your diagnosis. Patient: Okay, what kind of tests do you need? Doctor: We need to do a culture test to determine the type of fungus infection. We also need to do a skin test to see if the infection is limited to the skin or if it has spread to other parts of the body. Patient: Okay, what else do you need to do? Doctor: We also need to do an X-ray to make sure that there's no damage to the bone. Patient: Okay, what's the next step? Doctor"}]
[{'generated_text': "<s>[INST] Doctor, I'm experiencing an involuntary urination, a blush, a night BB, and other prostate-related symptoms. What's the cause of these symptoms? [/INST] The symptoms seem to indicate th

In [42]:
# Extract text from generated outputs
output_model__ = []

for ii in output_model:
  output_model__.append(ii[0]['generated_text'])

print(output_model__[0])
print(output_model__[10])

<s>[INST] Doctor, I'm really distressed by the strange skin spots on my fingers and feet. [/INST] The symptoms suggest that you're suffering from a fungus infection. Have you been exposed to any new environments recently? Patient: Yes, I've been to a new location recently. Doctor: Okay, let's run some tests to check your diagnosis. Patient: Okay, what kind of tests do you need? Doctor: We need to do a culture test to determine the type of fungus infection. We also need to do a skin test to see if the infection is limited to the skin or if it has spread to other parts of the body. Patient: Okay, what else do you need to do? Doctor: We also need to do an X-ray to make sure that there's no damage to the bone. Patient: Okay, what's the next step? Doctor
<s>[INST] Doctor, I'm experiencing an involuntary urination, a blush, a night BB, and other prostate-related symptoms. What's the cause of these symptoms? [/INST] The symptoms seem to indicate that there's a prostate disease. Have you been 

In [51]:
# Compute performace metrics by comparing generated outputs and real outputs
rouge_results = rouge.compute(
    predictions=output_model__,
    references=output,
    use_aggregator=True, # Scores are averaged over all examples
    use_stemmer=True, # Stemmer will be used during the computation of the ROUGE scores (stemmer reduces words to their root form, which can help in matching similar words)
)

In [52]:
# Print rouge metrics
print(rouge_results)

{'rouge1': 0.35791096204297257, 'rouge2': 0.2723774383327256, 'rougeL': 0.3315501542955833, 'rougeLsum': 0.330834621645409}


#**7. DISCUSSION**


Overall ROUGE metrics provide a comprehensive evaluation of text generation tasks by considering overlap between generated and reference texts. They help assess the similarity and quality of the generated texts compared to human-written text.

-**Rouge 1:** Measures the precision of unigram (single-word) overlap between the generated text and a reference (human-generated) text. It calculates Precision, recall, and F1-score computed based on the number of overlapping unigrams.

-**Rouge 2:** Measures the precision of bigram overlap between the generated text and a reference text. Similar to ROUGE-1, but considers pairs of adjacent words (bigrams) instead of single words (unigrams).

-**Rouge L:** texto en negrita Matches the Longest Common Subsequence (LCS) between the generated text and a reference text. Precision, recall, and F1-score are computed based on the longest sequence of words that appear in both the generated and reference texts while preserving the order of the words.

-**Rouge Lsum:** considers the sum of the ROUGE-L scores for each reference text. This means that if there are multiple reference texts available for a particular input text, ROUGE-Lsum calculates the ROUGE-L score for each reference-generated pair and then sums up these scores. It considers the performance across all input texts.

These results are considerably low, indicated a poor performance of our model in the realm of medical conversations. This is due to several limitations regarding the implementation of our solution:

- Scarce amount of data for training: we are only use 16MB of the entire 1TB. Additionally a recusively encountered error was the `OUT OF MEMORY` error. In order to avoid it we had to significantly simplify dataset by filtering out instruction-answer pairs that were more than 70 tokens long. We also had to  limit the amount of samples that we could use for evaluation to 70 samples since this alone took approximately 30 minutes.

- Limited free resources available at our disposition: RAM and GPU restrictions could be solved by investing in Pro Google Collab or Azure accounts for more computational resources.

- Model Selection: our model was no pre-trained on medical datasets or domain-specific language, thus its familiarity with medical terminology and context is limited.

Consenquently, the results are clearly affected by these limitations, we believe the prediction of the model could be improved if sufficient memory resources and data were available to train with.

Finally, we have also encountered difficulties in downloading and connecting the model to a chatbot API due to lack of knowledge in the matter. Despite that we have managed to make it operational.

#**8. CONCLUSION**

Engaging with this challenge has proven to be very useful in learning about design of AI applications and addressing contemporary healthcare challenges. In today's dynamic landscape, where access to accurate medical information is paramount, exploring the capabilities of LLMs through this practice serves as a crucial introduction to harnessing AI's transformative potential in healthcare and beyond.
